In [61]:
import ee
import geemap
import geopandas as gpd
import pandas as pd
import pandas as pd
import json
import configparser
import time

In [62]:
Map = geemap.Map()

In [63]:
config = configparser.ConfigParser()
config.read(r"./config.ini")

ee_collection = config.get("myvars", "ee_col")
path = config.get("myvars", "path")
band_ee = config.get("myvars", "band_ee")
cc_cover = config.get("myvars", "cc_cover")

folder = config.get("myvars", "folder")

In [89]:
dataset = ee.ImageCollection(ee_collection).filterDate('2017-01-01', '2017-12-31')
shpz = geemap.shp_to_ee(path)
dataset2 = dataset.mean().clip(shpz)
ndwi = dataset2.normalizedDifference(['B3', 'B5'])
Map.addLayer(ndwi,{'palette': ['red', 'yellow', 'green', 'cyan', 'blue']},'NDWI')
Map.centerObject(shpz,9)

In [127]:
ndwiThreshold = (ndwi.gte(0.3))
ndwiMask = ndwiThreshold.updateMask(ndwiThreshold);
Map.addLayer(
    ndwiThreshold,
    {'palette':['black', 'white']},
    'NDWI Binary Mask');
Map.addLayer(ndwiMask, {'palette':['blue']}, 'NDWI Mask');

In [119]:
Map

Map(bottom=468451.0, center=[18.91960278327717, 72.79986963757503], controls=(WidgetControl(options=['position…

In [120]:
pixel_area1 = ndwiMask.multiply(ee.Image.pixelArea())

Area = pixel_area1.reduceRegion(
    reducer = ee.Reducer.sum(),
    geometry = shpz,
    scale = 30,
    maxPixels = 1e9
).getInfo()

In [121]:
print(Area)

{'nd': 76352230.08828087}
